<a href="https://colab.research.google.com/github/butyusya/project_lbacn/blob/main/crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://butyusya:<PERSONAL_ACCESS_TOKEN>@github.com/butyusya/project_lbacn

Cloning into 'project_lbacn'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 11 (delta 1), reused 3 (delta 0), pack-reused 0
Receiving objects: 100% (11/11), 5.37 KiB | 5.37 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [2]:
!git config --global user.email
!git config --global user.name "butyusya"

In [3]:
!pip install fake_useragent

In [4]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import re

In [5]:
ua = UserAgent()
headers = {'User-Agent': ua.random}
session = requests.session()

In [6]:
def parse_episode(episode):
  block = {}
  info = episode.find_all('td')
  id = info[4].text.strip()
  block['ep_id'] = int(re.sub(r'\[\d+\]', '', id))
  block['season'] = block['ep_id'] // 100
  block['ep_href'] = info[2].find('a').attrs['href']
  block['ep_title'] = info[2].find('a').text
  return block

In [7]:
def parse_episode_info(block):
  url = 'https://miraculousladybug.fandom.com' + block['ep_href']
  req = session.get(url, headers={'User-Agent': ua.random})
  body = req.text
  text = body.split('Synopsis</span>')[2]
  first_div = text.split('<span class="mw-headline" id="Plot">Plot</span>')
  syn = first_div[0]
  porridge = BeautifulSoup(syn, 'html.parser')
  synopsis = porridge.find('p').text.strip()
  block['synopsis'] = re.sub(r'\[\d+\]', '', synopsis)
  other = first_div[1]
  second_div = other.split('<h2>')
  pl = second_div[0]
  porridge = BeautifulSoup(pl, 'html.parser')
  pl_pars = porridge.find_all('p')
  plot = ''
  for p in pl_pars:
    plot += p.text
  block['plot'] = re.sub(r'\[\d+\]', '', plot)
  url_scr = 'https://miraculousladybug.fandom.com' + block['ep_href']+ '/Transcript'
  req = session.get(url_scr, headers={'User-Agent': ua.random})
  body = req.text
  script = body.split('</td></tr></tbody></table></center>')[1].split('THE END')[0]
  porridge = BeautifulSoup(script, 'html.parser')
  block['script'] = porridge.text.replace('[ Theme Song ]', '').replace('[ Theme song ]', '').strip()
  return block

In [8]:
def parse_season(table, blocks):
  episodes = table.find_all('tr')
  for n in range(1, len(episodes)):
    try:
      block = parse_episode(episodes[n])
    except Exception as e:
      print(1, e, n)
    try:
      parse_episode_info(block)
    except Exception as e:
      print(2, e, n)
    try:
      blocks.append(block)
    except Exception as e:
      print(3, e, n)
  return blocks

In [9]:
blocks = []
for i in range(1, 6):
  try:
    url_one = 'https://miraculousladybug.fandom.com/wiki/Season_' + str(i)
    req = session.get(url_one, headers={'User-Agent': ua.random})
    body = req.text
    soup = BeautifulSoup(body, 'html.parser')
    tables = soup.find_all('table')
    table = tables[4]
    parse_season(table, blocks)
  except Exception as e:
    print(0, e, i)

In [10]:
new_table_query = """
CREATE TABLE episodes (
    ep_id INT,
    season INT,
    ep_href TEXT,
    ep_title TEXT,
    synopsis TEXT,
    plot TEXT,
    script TEXT,
    PRIMARY KEY (ep_id)
)
"""

In [11]:
import sqlite3

con = sqlite3.connect('project_lbacn/lbacn.db')
cur = con.cursor()
cur.execute(new_table_query)
con.commit()

In [12]:
for block in blocks:
  cur.execute("INSERT INTO episodes VALUES (:ep_id, :season, :ep_href, :ep_title, :synopsis, :plot, :script)", block)
con.commit()

In [13]:
def get_image(block):
  url = 'https://miraculousladybug.fandom.com' + block['ep_href']
  req = session.get(url, headers={'User-Agent': ua.random})
  body = req.text
  porridge = BeautifulSoup(body, 'html.parser')
  image_info = porridge.find('figure', {'class': 'pi-item pi-image'})
  image_url = image_info.find('a').attrs['href']
  img_data = requests.get(image_url).content
  with open('project_lbacn/static/'+str(block['ep_id'])+'.png', 'wb') as handler:
    handler.write(img_data)

In [15]:
for b in blocks:
  try:
    get_image(b)
  except Exception as e:
    print(b['ep_id'], e)

In [16]:
%cd project_lbacn
!git add lbacn.db
!git add static

/content/project_lbacn


In [17]:
!git commit -m 'db and images'

[main 72af00b] db and images
 132 files changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 lbacn.db
 create mode 100644 static/101.png
 create mode 100644 static/102.png
 create mode 100644 static/103.png
 create mode 100644 static/104.png
 create mode 100644 static/105.png
 create mode 100644 static/106.png
 create mode 100644 static/107.png
 create mode 100644 static/108.png
 create mode 100644 static/109.png
 create mode 100644 static/110.png
 create mode 100644 static/111.png
 create mode 100644 static/112.png
 create mode 100644 static/113.png
 create mode 100644 static/114.png
 create mode 100644 static/115.png
 create mode 100644 static/116.png
 create mode 100644 static/117.png
 create mode 100644 static/118.png
 create mode 100644 static/119.png
 create mode 100644 static/120.png
 create mode 100644 static/121.png
 create mode 100644 static/122.png
 create mode 100644 static/123.png
 create mode 100644 static/124.png
 create mode 100644 static/125.png
 create mode 10

In [18]:
!git push

Enumerating objects: 136, done.
Counting objects: 100% (136/136), done.
Delta compression using up to 2 threads
Compressing objects: 100% (135/135), done.
Writing objects: 100% (135/135), 455.23 MiB | 16.01 MiB/s, done.
Total 135 (delta 0), reused 0 (delta 0), pack-reused 0
To https://github.com/butyusya/project_lbacn
   63db297..72af00b  main -> main
